# HNSCC Machine Learning Pipeline
## Binary Classification: Primary Tumor vs. Normal Tissue

This notebook demonstrates a complete machine learning pipeline for classifying HNSCC samples as Primary Tumor or Normal tissue based on gene expression data.

---
## 1. Import Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    classification_report, roc_curve, auc
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print('Libraries imported successfully!')

---
## 2. Generate Placeholder Data

Since we're working without actual HNSCC data, we'll generate realistic placeholder data.
In production, you would load your expression matrix and metadata here.

In [ ]:
def generate_hnscc_data(n_samples=150, n_genes=500, random_state=42):
    """
    Generate placeholder HNSCC expression data.
    
    Parameters:
    - n_samples: Number of samples
    - n_genes: Number of genes
    - random_state: For reproducibility
    """
    np.random.seed(random_state)
    
    # Generate expression matrix
    X = pd.DataFrame(
        np.random.randn(n_samples, n_genes),
        columns=[f'ENSG{i:06d}' for i in range(n_genes)]
    )
    
    # Create imbalanced labels (more tumors than normal)
    n_normal = int(n_samples * 0.25)
    y = pd.Series([0] * n_normal + [1] * (n_samples - n_normal))
    y = y.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    return X, y

# Generate data
X, y = generate_hnscc_data(n_samples=150, n_genes=500)

print(f'Expression matrix shape: {X.shape}')
print(f'Sample labels shape: {y.shape}')
print(f'\nClass distribution:')
print(y.value_counts())
print(f'\nClass proportions:')
print(y.value_counts(normalize=True))

---
## 3. Data Preprocessing

### 3.1 Train-Test Split

In [ ]:
# Stratified train-test split (maintains class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f'Training set: {X_train.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')
print(f'\nTraining set class distribution:')
print(y_train.value_counts())
print(f'\nTest set class distribution:')
print(y_test.value_counts())

### 3.2 Feature Scaling

In [ ]:
# Feature scaling (important for ElasticNet)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print('Feature scaling completed!')
print(f'Training set - Mean: {X_train_scaled.mean().mean():.4f}, Std: {X_train_scaled.std().mean():.4f}')
print(f'Test set - Mean: {X_test_scaled.mean().mean():.4f}, Std: {X_test_scaled.std().mean():.4f}')

---
## 4. Model Training

### 4.1 ElasticNet (Logistic Regression with L1/L2)

In [ ]:
# ElasticNet model
elasticnet = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    l1_ratio=0.5,
    max_iter=1000,
    random_state=42
)

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
en_cv_scores = cross_val_score(elasticnet, X_train_scaled, y_train, cv=cv, scoring='roc_auc')

print('ElasticNet Cross-Validation Results:')
print(f'AUC scores: {en_cv_scores}')
print(f'Mean AUC: {en_cv_scores.mean():.4f} ± {en_cv_scores.std():.4f}')

# Train on full training set
elasticnet.fit(X_train_scaled, y_train)
print('\nElasticNet model trained!')

### 4.2 Random Forest

In [ ]:
# Random Forest model
random_forest = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

# Cross-validation
rf_cv_scores = cross_val_score(random_forest, X_train_scaled, y_train, cv=cv, scoring='roc_auc')

print('Random Forest Cross-Validation Results:')
print(f'AUC scores: {rf_cv_scores}')
print(f'Mean AUC: {rf_cv_scores.mean():.4f} ± {rf_cv_scores.std():.4f}')

# Train on full training set
random_forest.fit(X_train_scaled, y_train)
print('\nRandom Forest model trained!')

---
## 5. Model Evaluation

### 5.1 Predictions on Test Set

In [ ]:
# ElasticNet predictions
en_pred = elasticnet.predict(X_test_scaled)
en_pred_proba = elasticnet.predict_proba(X_test_scaled)[:, 1]

# Random Forest predictions
rf_pred = random_forest.predict(X_test_scaled)
rf_pred_proba = random_forest.predict_proba(X_test_scaled)[:, 1]

print('Predictions generated!')

### 5.2 Performance Metrics

In [ ]:
# Calculate metrics
models = ['ElasticNet', 'Random Forest']
predictions = [en_pred, rf_pred]
predictions_proba = [en_pred_proba, rf_pred_proba]

results = {}

for model_name, pred, pred_proba in zip(models, predictions, predictions_proba):
    accuracy = accuracy_score(y_test, pred)
    auc_score = roc_auc_score(y_test, pred_proba)
    cm = confusion_matrix(y_test, pred)
    
    results[model_name] = {
        'accuracy': accuracy,
        'auc': auc_score,
        'confusion_matrix': cm
    }
    
    print(f'\n{model_name}:')
    print(f'  Test Accuracy: {accuracy:.4f}')
    print(f'  Test AUC: {auc_score:.4f}')
    print(f'  Confusion Matrix:\n{cm}')
    print(f'\n  Classification Report:')
    print(classification_report(y_test, pred, target_names=['Normal', 'Tumor']))

### 5.3 Confusion Matrices Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (model_name, pred) in enumerate(zip(models, predictions)):
    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
               xticklabels=['Normal', 'Tumor'],
               yticklabels=['Normal', 'Tumor'],
               cbar=False)
    axes[idx].set_title(f'{model_name}\nConfusion Matrix', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('True Label', fontsize=11)
    axes[idx].set_xlabel('Predicted Label', fontsize=11)

plt.tight_layout()
plt.show()

### 5.4 ROC Curves

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

for model_name, pred_proba in zip(models, predictions_proba):
    fpr, tpr, _ = roc_curve(y_test, pred_proba)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.3f})', linewidth=2.5)

ax.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1.5)
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves - HNSCC Tumor Classification', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 6. Feature Importance Analysis

### 6.1 Extract Feature Importance

In [ ]:
# ElasticNet coefficients
en_importance = pd.DataFrame({
    'gene': X.columns,
    'coefficient': elasticnet.coef_[0],
    'abs_coefficient': np.abs(elasticnet.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

# Random Forest feature importances
rf_importance = pd.DataFrame({
    'gene': X.columns,
    'importance': random_forest.feature_importances_
}).sort_values('importance', ascending=False)

print('Feature Importance Extracted!')
print(f'\nTop 10 Features - ElasticNet:')
print(en_importance.head(10))
print(f'\nTop 10 Features - Random Forest:')
print(rf_importance.head(10))

### 6.2 Feature Importance Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# ElasticNet
en_top20 = en_importance.head(20)
colors = ['red' if x < 0 else 'blue' for x in en_top20['coefficient']]
axes[0].barh(range(len(en_top20)), en_top20['coefficient'], color=colors)
axes[0].set_yticks(range(len(en_top20)))
axes[0].set_yticklabels(en_top20['gene'], fontsize=9)
axes[0].set_xlabel('Coefficient', fontsize=11)
axes[0].set_title('Top 20 Features - ElasticNet', fontsize=12, fontweight='bold')
axes[0].invert_yaxis()
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)

# Random Forest
rf_top20 = rf_importance.head(20)
axes[1].barh(range(len(rf_top20)), rf_top20['importance'], color='steelblue')
axes[1].set_yticks(range(len(rf_top20)))
axes[1].set_yticklabels(rf_top20['gene'], fontsize=9)
axes[1].set_xlabel('Importance', fontsize=11)
axes[1].set_title('Top 20 Features - Random Forest', fontsize=12, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

---
## 7. Summary and Conclusions

### Model Performance Summary

In [ ]:
summary_data = {
    'Model': ['ElasticNet', 'Random Forest'],
    'CV AUC Mean': [en_cv_scores.mean(), rf_cv_scores.mean()],
    'CV AUC Std': [en_cv_scores.std(), rf_cv_scores.std()],
    'Test AUC': [results['ElasticNet']['auc'], results['Random Forest']['auc']],
    'Test Accuracy': [results['ElasticNet']['accuracy'], results['Random Forest']['accuracy']]
}

summary_df = pd.DataFrame(summary_data)
print('\nModel Performance Summary:')
print(summary_df.to_string(index=False))

print('\n' + '='*60)
print('KEY FINDINGS:')
print('='*60)
print(f'\n1. Best Performing Model: Random Forest')
print(f'   - Test AUC: {results["Random Forest"]["auc"]:.4f}')
print(f'   - Test Accuracy: {results["Random Forest"]["accuracy"]:.4f}')
print(f'\n2. Feature Stability:')
print(f'   - ElasticNet identified {len(en_importance)} discriminative genes')
print(f'   - Random Forest identified {len(rf_importance)} important genes')
print(f'\n3. Top Candidate Genes (Consensus):')
print(f'   - {en_importance.iloc[0]["gene"]}')
print(f'   - {rf_importance.iloc[0]["gene"]}')
print('\n' + '='*60)

---
## 8. Next Steps

1. **Biological Validation**: Verify top features against known HNSCC biology
2. **External Validation**: Test on independent HNSCC cohorts
3. **Clinical Integration**: Incorporate clinical variables (TNM stage, HPV status)
4. **Survival Analysis**: Extend to predict patient outcomes
5. **Pathway Analysis**: Perform enrichment analysis on top genes

---
*Pipeline completed successfully!*